Fernando Boffi - PSet 1 - Macroeconomics

In [ ]:
using Pkg

In [273]:
Pkg.add("StatsBase")

   Resolving package versions...
    Updating `C:\Users\joseb\.julia\environments\v1.7\Project.toml`
  [2913bbd2] + StatsBase v0.33.21
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Manifest.toml`


In [251]:
Pkg.add("SpecialFunctions")

   Resolving package versions...
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Manifest.toml`


In [274]:
using Distributions, Parameters, PrettyTables, Statistics, StatsBase

#### Question 1

In [252]:
using SpecialFunctions

In [253]:
std_norm_cdf(x::T) where {T <: Real} = 0.5 * erfc(-x/sqrt(2))
std_norm_cdf(x::Array{T}) where {T <: Real} = 0.5 .* erfc(-x./sqrt(2))

std_norm_cdf (generic function with 2 methods)

In [408]:
function tauchen(N, ρ, σ,  n_std)   # Tauchen Method
    # Get discretized space
    Y_N = n_std * sqrt(σ^2 / (1 - ρ^2))
    y = range(-Y_N, stop=Y_N, length=N)
    d = y[2] - y[1]

    # Get transition probabilities
    Π = zeros(N, N)
    for row = 1:N
        # Do end points first
        Π[row, 1] = std_norm_cdf((y[1] - ρ*y[row] + d/2) / σ)
        Π[row, N] = 1 - std_norm_cdf((y[N] - ρ*y[row] - d/2) / σ)

        # fill in the middle columns
        for col = 2:N-1
            Π[row, col] = (std_norm_cdf((y[col] - ρ*y[row] + d/2) / σ) -
                           std_norm_cdf((y[col] - ρ*y[row] - d/2) / σ))
        end
    end
    return (Π,collect(y))
    
end

tauchen (generic function with 1 method)

In [19]:
Pkg.add("Optim");

   Resolving package versions...
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Manifest.toml`


In [18]:
Pkg.add("NLopt");

   Resolving package versions...
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Manifest.toml`


In [558]:
(P_t, range_t) = tauchen(3,0.2,0.4,3)

([0.17916323337444012 0.8047923996364443 0.01604436698911571; 0.06289321231947004 0.8742135753610598 0.0628932123194701; 0.016044366989115655 0.8047923996364442 0.17916323337444018], [-1.2247448713915892, 0.0, 1.2247448713915892])

In [674]:
range_t

3-element Vector{Float64}:
 -1.2247448713915892
  0.0
  1.2247448713915892

In [560]:
P_t

3×3 Matrix{Float64}:
 0.179163   0.804792  0.0160444
 0.0628932  0.874214  0.0628932
 0.0160444  0.804792  0.179163

#### Question 2

In [2]:
function _rouwenhorst2(ρ, σ::Real, n::Integer)
    
    σ_y = σ / sqrt(1-ρ^2)
    p  = (1+ρ)/2
    Θ = [p 1-p; 1-p p]
    ψ = sqrt(n-1) * σ_y
    q  = (1+ρ)/2
    
    if n == 2
        return [-ψ, ψ],  [p 1-p; 1-q q]
    else
        _, θ_nm1 = _rouwenhorst2(ρ, σ, n-1)
        θN = p    *[θ_nm1 zeros(n-1, 1); zeros(1, n)] +
             (1-p)*[zeros(n-1, 1) θ_nm1; zeros(1, n)] +
             q    *[zeros(1, n); zeros(n-1, 1) θ_nm1] +
             (1-q)*[zeros(1, n); θ_nm1 zeros(n-1, 1)]

        θN[2:end-1, :] ./= 2

        return collect(range(-ψ, stop=ψ, length=n)), θN
    end
end


_rouwenhorst2 (generic function with 1 method)

In [8]:
(Range, Prob) = _rouwenhorst2(0.3, 0.5, 3)

([-0.7412493166611012, 0.0, 0.7412493166611012], [0.42250000000000004 0.45499999999999996 0.12249999999999998; 0.22749999999999998 0.545 0.22749999999999998; 0.12249999999999998 0.45499999999999996 0.42250000000000004])

In [9]:
Prob

3×3 Base.Matrix{Float64}:
 0.4225  0.455  0.1225
 0.2275  0.545  0.2275
 0.1225  0.455  0.4225

In [10]:
Range

3-element Vector{Float64}:
 -0.7412493166611012
  0.0
  0.7412493166611012

#### Question 3

In [565]:
function simulate_ar1(phi::Any, n::Any, σ) # function that computes the AR1

y = [0.0 for i = 1:n]

noise = rand(Normal(0, σ), n)

for i in 1:(n-1)
y[1]=0
y[i+1] = phi*y[i] + noise[i] 
end

return y 
end

simulate_ar1 (generic function with 2 methods)

In [566]:
y_ar1=simulate_ar1(0.2, 1000, 0.4);

In [675]:
y_ar1

1000-element Vector{Float64}:
  0.0
 -0.6082165417654758
  0.6092855333875722
  5.58505080790056e-5
  0.14543521879420365
  0.7657150507879649
 -0.01049918243582737
  0.27479720482832315
 -0.06251485269686058
 -0.050056821014214894
  0.0870225153280586
 -0.47625123739242403
 -0.14792345863633505
  ⋮
  0.021783373041462172
  0.36486560587541145
 -0.03690048626223287
  0.3850329665739796
 -0.16240437636228516
  0.5126921434437027
 -0.10379873928311881
  0.656610657978269
 -0.10503417253339203
  0.4023380992204816
  0.478119785437581
 -0.30903347597119696

In [568]:
Pkg.add("QuantEcon")

   Resolving package versions...
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\joseb\.julia\environments\v1.7\Manifest.toml`


In [265]:
using QuantEcon

In [649]:
mc = MarkovChain(P_t)
X_t = simulate_indices(mc, 1000, init=2); # simulates generates a draw from a Markov process

In [650]:
X_t;# the output is a position vector. X_t refers to Tauchen

In [651]:
ysim_t=range_t[X_t];

In [676]:
ysim_t

1000-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
 -1.2247448713915892
  0.0
  0.0
  0.0
  1.2247448713915892
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
 -1.2247448713915892
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [653]:
mc = MarkovChain(P_r)
X_r = simulate_indices(mc, 1000, init=2); # simulates generates a draw from a Markov process (rouwenhorst)
# init=2 means that the initial C

In [654]:
X_r; #rouwenhorst

In [655]:
ysim_r=range_r[X_r];

In [656]:
tab_out = ["true AR1" mean(y_ar1) var(y_ar1) autocor(y_ar1,[1]) quantile(y_ar1, [0.5 0.1 0.9]);
           "tauchen" mean(ysim_t) var(ysim_t) autocor(ysim_t,[1]) quantile(ysim_t, [0.5 0.1 0.9]);
           "rouwen" mean(ysim_r) var(ysim_r) autocor(ysim_r,[1]) quantile(ysim_r, [0.5 0.1 0.9])]
header = ["method", "mean", "variance", "autocorrelation", "median", "P10", "P90"]
pretty_table(tab_out; header)

┌──────────┬───────────┬──────────┬─────────────────┬────────────┬───────────┬──────────┐
│   method │      mean │ variance │ autocorrelation │     median │       P10 │      P90 │
├──────────┼───────────┼──────────┼─────────────────┼────────────┼───────────┼──────────┤
│ true AR1 │ 0.0344152 │ 0.165547 │         0.23371 │ 0.00303175 │ -0.482077 │ 0.582212 │
│  tauchen │ 0.0110227 │ 0.187566 │        0.127434 │        0.0 │       0.0 │      0.0 │
│   rouwen │ 0.0109697 │ 0.168381 │        0.245042 │        0.0 │  -0.57735 │  0.57735 │
└──────────┴───────────┴──────────┴─────────────────┴────────────┴───────────┴──────────┘


Altough the N=3 (very few states), With low persistence, there is no huge difference between the two methods (the difference we see in the table regarding the variance and the autocorrelation are sesntive to different draws from the Markov Chains (also t=1000 is not much).

## Question 4

In [625]:
function tauchenplus(N,ρ)
  (Pt, yt) = tauchen(N,ρ,0.4,3)
   mct = MarkovChain(Pt)
   xt = simulate_indices(mct, 1000, init=ceil(Int, N/2)); # simulates generates a draw from a Markov process using  T method
   ysim_t_ρ=yt[xt]
return ysim_t_ρ
end

tauchenplus (generic function with 1 method)

In [626]:
ysim_t_10=tauchenplus(10,0.2);

In [627]:
function rouwennplus(N,ρ)
  (yr, Pr) = _rouwenhorst2(ρ, 0.4, N)
  mcr = MarkovChain(Pr)
  xr = simulate_indices(mcr, 1000, init=ceil(Int, N/2)); # simulates generates a draw from a Markov process
  ysim_r_ρ=yr[xr]
return  ysim_r_ρ
end

rouwennplus (generic function with 1 method)

In [628]:
ysim_r_10=rouwennplus(10,0.2);

In [629]:
tab_out = ["true AR1" mean(y_ar1) var(y_ar1) autocor(y_ar1,[1]) quantile(y_ar1, [0.5 0.1 0.9]);
           "tauchen" mean(ysim_t_10) var(ysim_t_10) autocor(ysim_t_10,[1]) quantile(ysim_t_10, [0.5 0.1 0.9]);
           "rouwen" mean(ysim_r_10) var(ysim_r_10) autocor(ysim_r_10,[1]) quantile(ysim_r_10, [0.5 0.1 0.9])]
header = ["method", "mean", "variance", "autocorrelation", "median", "P10", "P90"]
pretty_table(tab_out; header)

┌──────────┬───────────┬──────────┬─────────────────┬────────────┬───────────┬──────────┐
│   method │      mean │ variance │ autocorrelation │     median │       P10 │      P90 │
├──────────┼───────────┼──────────┼─────────────────┼────────────┼───────────┼──────────┤
│ true AR1 │ 0.0344152 │ 0.165547 │         0.23371 │ 0.00303175 │ -0.482077 │ 0.582212 │
│  tauchen │ 0.0195959 │ 0.159331 │        0.168692 │   0.136083 │ -0.408248 │ 0.408248 │
│   rouwen │ 0.0236784 │ 0.155447 │        0.175184 │   0.136083 │ -0.408248 │ 0.408248 │
└──────────┴───────────┴──────────┴─────────────────┴────────────┴───────────┴──────────┘


With N=10, and still With low persistence, there is no huge difference between the two methods. In both cases using N=10 improves the approximation of an AR1 in both methods. As we can see in the table the estimation of the variance is very precise and there is an improvment regarding the estimation of the autocorrelation with respect to the case with N=3

## Question 5

# ρ=0.7 

In [666]:
ysim_t_1=tauchenplus(3,0.7);
ysim_r_1=rouwennplus(3,0.7);
yar1=simulate_ar1(0.7, 1000, 0.4);
ysim_t_2=tauchenplus(10,0.7);
ysim_r_2=rouwennplus(10,0.7);

In [667]:
ysim_t_1

1000-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
 -1.680336100833612
 -1.680336100833612
 -1.680336100833612
 -1.680336100833612
 -1.680336100833612
 -1.680336100833612
 -1.680336100833612
 -1.680336100833612
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [665]:
tab_out = ["true AR1" 0 mean(yar1) var(yar1) autocor(yar1,[1]) quantile(yar1, [0.5 0.1 0.9]);
           "Tauchen" 3 mean(ysim_t_1) var(ysim_t_1) autocor(ysim_t_1,[1]) quantile(ysim_t_1, [0.5 0.1 0.9]);
           "rouwen" 3 mean(ysim_r_1) var(ysim_r_1) autocor(ysim_r_1,[1]) quantile(ysim_r_1, [0.5 0.1 0.9]);
           "tauchen" 10 mean(ysim_t_2) var(ysim_t_2) autocor(ysim_t_2,[1]) quantile(ysim_t_2, [0.5 0.1 0.9]);
           "rouwen" 10 mean(ysim_r_2) var(ysim_r_2) autocor(ysim_r_2,[1]) quantile(ysim_r_2, [0.5 0.1 0.9])]
header = ["method", "N", "mean", "variance", "autocorrelation", "median", "P10", "P90"]
pretty_table(tab_out; header)

┌──────────┬────┬─────────────┬──────────┬─────────────────┬────────────┬───────────┬──────────┐
│   method │  N │        mean │ variance │ autocorrelation │     median │       P10 │      P90 │
├──────────┼────┼─────────────┼──────────┼─────────────────┼────────────┼───────────┼──────────┤
│ true AR1 │  0 │  -0.0469864 │ 0.281148 │        0.662018 │ -0.0406872 │ -0.747202 │ 0.641441 │
│  Tauchen │  3 │  0.00168034 │  0.37025 │        0.778624 │        0.0 │       0.0 │      0.0 │
│   rouwen │  3 │  -0.0213872 │ 0.328027 │         0.72428 │        0.0 │ -0.792118 │ 0.792118 │
│  tauchen │ 10 │ -0.00261386 │  0.34418 │        0.714213 │  -0.186704 │ -0.597453 │  0.93352 │
│   rouwen │ 10 │  -0.0433153 │ 0.264986 │        0.691914 │  -0.186704 │ -0.560112 │ 0.560112 │
└──────────┴────┴─────────────┴──────────┴─────────────────┴────────────┴───────────┴──────────┘


# ρ=0.9 

In [668]:
ysim_t_1=tauchenplus(3,0.9);
ysim_r_1=rouwennplus(3,0.9);
yar1=simulate_ar1(0.9, 1000, 0.4);
ysim_t_2=tauchenplus(10,0.9);
ysim_r_2=rouwennplus(10,0.9);

In [669]:
ysim_t_1;

In [670]:
tab_out = ["true AR1" 0 mean(yar1) var(yar1) autocor(yar1,[1]) quantile(yar1, [0.5 0.1 0.9]);
           "Tauchen" 3 mean(ysim_t_1) var(ysim_t_1) autocor(ysim_t_1,[1]) quantile(ysim_t_1, [0.5 0.1 0.9]);
           "rouwen" 3 mean(ysim_r_1) var(ysim_r_1) autocor(ysim_r_1,[1]) quantile(ysim_r_1, [0.5 0.1 0.9]);
           "tauchen" 10 mean(ysim_t_2) var(ysim_t_2) autocor(ysim_t_2,[1]) quantile(ysim_t_2, [0.5 0.1 0.9]);
           "rouwen" 10 mean(ysim_r_2) var(ysim_r_2) autocor(ysim_r_2,[1]) quantile(ysim_r_2, [0.5 0.1 0.9])]
header = ["method", "N", "mean", "variance", "autocorrelation", "median", "P10", "P90"]
pretty_table(tab_out; header)

┌──────────┬────┬────────────┬──────────┬─────────────────┬───────────┬───────────┬──────────┐
│   method │  N │       mean │ variance │ autocorrelation │    median │       P10 │      P90 │
├──────────┼────┼────────────┼──────────┼─────────────────┼───────────┼───────────┼──────────┤
│ true AR1 │  0 │ -0.0464333 │  0.70736 │        0.873928 │ 0.0139644 │  -1.08714 │ 0.926286 │
│  Tauchen │  3 │   0.553351 │  1.21839 │        0.994773 │       0.0 │       0.0 │  2.75299 │
│   rouwen │  3 │  -0.016871 │ 0.844349 │        0.874235 │       0.0 │  -1.29777 │  1.29777 │
│  tauchen │ 10 │  -0.226357 │  1.17604 │        0.912307 │ -0.305888 │  -1.52944 │ 0.917663 │
│   rouwen │ 10 │  -0.115014 │ 0.754029 │        0.895799 │ -0.305888 │ -0.917663 │ 0.917663 │
└──────────┴────┴────────────┴──────────┴─────────────────┴───────────┴───────────┴──────────┘


# ρ=0.98 

In [671]:
ysim_t_1=tauchenplus(3,0.98);
ysim_r_1=rouwennplus(3,0.98);
yar1=simulate_ar1(0.98, 1000, 0.4);
ysim_t_2=tauchenplus(10,0.98);
ysim_r_2=rouwennplus(10,0.98);

In [672]:
yar1;

In [673]:
tab_out = ["true AR1" 0 mean(yar1) var(yar1) autocor(yar1,[1]) quantile(yar1, [0.5 0.1 0.9]);
           "Tauchen" 3 mean(ysim_t_1) var(ysim_t_1) autocor(ysim_t_1,[1]) quantile(ysim_t_1, [0.5 0.1 0.9]);
           "rouwen" 3 mean(ysim_r_1) var(ysim_r_1) autocor(ysim_r_1,[1]) quantile(ysim_r_1, [0.5 0.1 0.9]);
           "tauchen" 10 mean(ysim_t_2) var(ysim_t_2) autocor(ysim_t_2,[1]) quantile(ysim_t_2, [0.5 0.1 0.9]);
           "rouwen" 10 mean(ysim_r_2) var(ysim_r_2) autocor(ysim_r_2,[1]) quantile(ysim_r_2, [0.5 0.1 0.9])]
header = ["method", "N", "mean", "variance", "autocorrelation", "median", "P10", "P90"]
pretty_table(tab_out; header)

┌──────────┬────┬───────────┬──────────┬─────────────────┬───────────┬───────────┬─────────┐
│   method │  N │      mean │ variance │ autocorrelation │    median │       P10 │     P90 │
├──────────┼────┼───────────┼──────────┼─────────────────┼───────────┼───────────┼─────────┤
│ true AR1 │  0 │ -0.629682 │  3.95962 │        0.980101 │ -0.803109 │  -2.92525 │ 2.38134 │
│  Tauchen │  3 │       0.0 │      0.0 │             NaN │       0.0 │       0.0 │     0.0 │
│   rouwen │  3 │  0.466199 │  4.05338 │        0.975999 │       0.0 │  -2.84268 │ 2.84268 │
│  tauchen │ 10 │    1.7166 │   2.8923 │         0.96709 │   2.01008 │ -0.670025 │ 3.35013 │
│   rouwen │ 10 │  0.986277 │  4.16723 │        0.980793 │  0.670025 │  -2.01008 │ 4.69018 │
└──────────┴────┴───────────┴──────────┴─────────────────┴───────────┴───────────┴─────────┘


# Exercise 6

As we can see in point 5 the Rouwenhorst method is more occurate for highly persistent processes (the estimation of the variance and autocorrelations is more accurate). In the three cases (rho=0.7, rho=0.90 and rho=0.98) the approximation of the AR1 is better using the Rouwenhorst method (variance and autocorrelation closer to the true AR1)

Also note that the Tauchen method with N=3 and rho=0.98 has no result. This occurs because we choose the initial value of the Markov process to be the median of the states (in the case n=3 this initial state is 0). Therefore, since rho=0.98 (there is a lot of persistance) the process stays at the initial state. We can change this result by changing the starting point (such as a radom draw from all states) but by doing that we pay the cost of being inconsistent with the choice of the initial value of the AR1 (it starts in the mean=0) and still we do not have an accurate approximation (variance too high). Therefore, the Tauchen method is not an accurate approximation when the AR1 is persistent.

Regarding the difference between N=10 and N=3 we can see that the approximation of the Rouwenhorst method improves when N increase. As we pointed out the performance of the Tauchen method is not accurate when persistence is high.

In the case with low persistence (rho=0.2) we can see that there is an improvment in the approximation in both methods (both work well) when N increases.